In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

C:\Users\RAVI GUPTA\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [27]:
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [28]:
akshar_image_folders = os.listdir("Devanagari\Train")

In [29]:
list_of_images = []

list_of_labels = []

label_index = 0

for akshar_folder in akshar_image_folders:
    
    each_folder_path = os.path.join("Devanagari\Train",
                                   akshar_folder)
    
    images = os.listdir(each_folder_path)
    
    img_shape = plt.imread(os.path.join(each_folder_path,images[0])).shape
    
    blank_array = np.zeros((len(images),(img_shape[0]*img_shape[1]))) 
    
    blank_label_array = label_index*(np.ones((len(images),1)))
    
    index = 0
    
    for image in images:
        
        img_in_array = plt.imread(os.path.join(each_folder_path,image))
        
        blank_array[index] = img_in_array.reshape(1,(img_in_array.shape[0]*img_in_array.shape[1]))
        
        index += 1
        
    list_of_images.append(blank_array)
    
    list_of_labels.append(blank_label_array)
    
    label_index += 1

In [30]:
tuple_of_label_arrays = tuple(list_of_labels)

tuple_of_arrays = tuple(list_of_images)

stacked_up_images = np.concatenate(tuple_of_arrays)

print(stacked_up_images.shape)

stacked_up_labels = np.concatenate(tuple_of_label_arrays)

print(stacked_up_labels.shape)

stacked_up_labels = np.int32(stacked_up_labels)

rawdata = pd.DataFrame(stacked_up_images)

(61200, 1024)
(61200, 1)


In [31]:
#---one hot encoding of labels
one_hot_labels = np.eye(len(akshar_image_folders),
                        len(akshar_image_folders))[stacked_up_labels[:,0]]

In [32]:
#--Importing keras model and necessary classes
from keras.models import Model
from keras.layers import Activation,Conv2D,Dense,MaxPool2D,Input,Flatten,BatchNormalization,Dropout
# from tensorflow.keras.callbacks import TensorBoard

In [76]:
#--input layer for entering of given data
inp_layer = Input(shape=(32,32,1))

In [77]:
#--Creating two convulational layer and before entering of data into next layer we applying batch normalization technique 
#  which improve accuracy drastically

layer = Conv2D(filters=10,kernel_size=[5,5],activation=tf.nn.relu) (inp_layer)

layer = BatchNormalization()(layer)

layer = Conv2D(filters=10,kernel_size=[5,5],activation=tf.nn.relu) (layer)

layer = BatchNormalization()(layer)


In [78]:
#--maxpooling the output of convulational layer and before the feeding into Fully Connected Layer
#--and this technique help to reduce the no. units of neurons in any layer by downsampling

layer = MaxPool2D(strides=(2,2)) (layer)

In [79]:
#--streching a image to single row

layer = Flatten() (layer)

In [80]:
#--Creating Two Fully Connected Hidden layer and also use Batch Normalization Techinque

layer = Dense(units=16*16*10,activation=tf.nn.relu) (layer)

layer = BatchNormalization()(layer)

# layer = Dropout(rate=0.10)(layer)

layer = Dense(units=16*16*10,activation=tf.nn.relu) (layer)

layer = BatchNormalization()(layer)

# layer = Dropout(rate = 0.10)(layer)

In [81]:
#--Ouput layer which contain 36 neurons for 36 classes

layer = Dense(units=36,activation=tf.nn.softmax) (layer)

In [82]:
CNN = Model(inp_layer,layer)

In [83]:
#-- compiling the graph which is made above
# tensorboard = TensorBoard(log_dir="logs/{}".format('CNN_On_Devanagari_By_Keras'))

CNN.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [84]:
# stacked_up_images = stacked_up_images/255   --not effective technique in NN ..i think it generally reduce the variability of values which decrease the accuracy

reshaped_data = np.reshape(stacked_up_images,(-1,32,32,1))

In [85]:
CNN.fit(x=reshaped_data,y=one_hot_labels,batch_size=500,epochs=2,)

Epoch 1/2
61200/61200 [==============================] - 40s 647us/step - loss: 0.6541 - acc: 0.8546
Epoch 2/2
61200/61200 [==============================] - 37s 607us/step - loss: 0.1358 - acc: 0.9616


# Save Trained Model

In [90]:
from keras import models
models.Model.save(CNN,filepath = "CNN_On_Devanagari.h5")

## OR

In [ ]:
CNN.save(filepath = "CNN_On_Devanagari_2.h5")

# Load Pre_Trained Model

In [91]:
cnn = tf.keras.models.load_model("CNN_On_Devanagari.h5")  #--it return the tensor object of your trained model

cnn.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy']) #--necessary step 
cnn.fit(x=reshaped_data,y=one_hot_labels,batch_size=500,epochs=2)  # it is used to improve further the trained model

Epoch 1/2
61200/61200 [==============================] - 32s 521us/step - loss: 0.0752 - acc: 0.9767
Epoch 2/2
61200/61200 [==============================] - 29s 482us/step - loss: 0.0448 - acc: 0.9862


# Batch Normalization

The general use case is to use BN between the linear and non-linear layers in your network, because it normalizes the input to your activation function, so that you're centered in the linear section of the activation function (such as Sigmoid).

the point is that we're trying to normalize the inputs to a layer, so it should always go immediately before the next layer in the network